# Simple implementation af Kalman filteret
\begin{align*}
    \intertext{kinematic and observations model}
    \mathbf{x}[n]&=\Phi \mathbf{x}[n-1] + \mathbf{u}[n], \\
    \mathbf{z}[n]&=\mathbf{x}[n] + \mathbf{w}[n],\\
    \intertext{the prediction and prediction mean square error,}
    \hat{\mathbf{x}}[n|n-1]&=\Phi\hat{\mathbf{x}}[n-1|n-1], \\
    M[n|n-1]&=\Phi M[n-1|n-1]\Phi^\top + S_{\mathbf{u}}, \\
    \intertext{the Kalman gain,}
    K[n]&=M[n|n-1](S_{\mathbf{w}}+M[n|n-1])^{-1},\\
    \intertext{and the correction and estimate mean square error,}
    \hat{\mathbf{x}}[n|n]&=\hat{\mathbf{x}}[n|n-1]+K[n](\mathbf{z}[n]-\hat{\mathbf{x}}[n|n-1]),\\
    M[n|n]&=(I-K[n])M[n|n-1].
\end{align*}

In [1]:
import numpy as np
import matplotlib.pyplot as plt


distance = np.load("sat_2_100deg/distance.npy")
distances = np.load("sat_2_100deg/distances.npy")
velocity = np.load("sat_2_100deg/velocity.npy")
velocities = np.load("sat_2_100deg/velocties.npy")

In [13]:
class Kalman:
    z = []

    x_predictions = []
    x_corrections = []

    M_predictions = []
    M_corrections = []

    K = []

    def __init__(self, phi, S_u, S_w, x_init_guess, M_init_guess, x):
        # Save filter constants and data
        self.phi = phi
        self.S_u = S_u
        self.S_w = S_w
        self.dim = x.shape
        self.x = x

        # Save initial predicitons and initial measurement
        self.x_predictions.append(x_init_guess)
        self.M_predictions.append(M_init_guess)
        print(self.dim)
        self.z.append(x[0] + np.random.normal(np.zeros(self.dim), S_w, self.dim))


    def __make_guess(self):
        # Create guess of the next step
        x_guess = self.phi*self.x_predictions[-1]
        M_guess = self.phi*self.M_predictions[-1]*self.phi.T + self.S_u

        # Append the guess
        self.x_predictions.append(x_guess)
        self.M_predictions.append(M_guess)

        # create new observation
        n = len(self.x_predictions)
        obs = self.x[n-1] + np.random.normal(np.zeros(self.dim), self.S_w, self.dim)
        self.z.append(obs)


    def __kalman_gain(self):
        # Calculate and append the Kalman gain
        gain = self.M_predictions[-1]*np.linalg.inv(self.S_w+self.M_predictions[-1])
        self.K.append(gain)


    def __correction(self):
        # Calculate the correciton step of the process
        x_correct = self.x_predictions[-1]+self.K[-1]*(self.z[-1]-self.x_predictions[-1])
        M_correct = (np.eye(self.dim)-self.K[-1])*self.M_predictions[-1]

        # Append the corrections
        self.x_corrections.append(x_correct)
        self.M_corrections.append(M_correct)


    def run_sim(self):
        while  len(self.x_predictions) <= len(self.x):
            self.__make_guess()
            self.__kalman_gain()
            self.__correction()

In [3]:
"""
    Her laver vi state vektor x_state og vores transition matrix. Bemærk at F er tidsafhængig men vi
    lader den bare være bestemt ud fra vores initial parameters
"""
x_state = np.hstack([distances, velocities])

# Skab vores force F matrix
r_i, r_j, r_k = [x_state[0, i] for i in range(3)]
r = distance[0]
mu = 3.986004418e14 # wiki "standard gravitational parameter"

# F kan findes på side 11 i thesis
F1, F2, F4 = np.zeros((3, 3)), np.eye(3), np.zeros((3, 3))
F3 = np.asanyarray([[-mu/(r**3) + (3*mu*r_i)**2/(r**5), (3*mu*r_i*r_j)/(r**5), (3*mu*r_i*r_k)/(r**5)],
                    [(3*mu*r_i*r_j)/(r**5), -mu/r**3 + (3*mu*r_j**2)/(r**5), (3*mu*r_j*r_k)/(r**5)],
                    [(3*mu*r_i*r_k)/(r**5), (3*mu*r_j*r_k)/(r**5), -(mu)/(r**3) + (3*mu*r_k**2)/(r**5)]])
F_top = np.concatenate((F1, F2), axis=1)
F_bot = np.concatenate((F3, F4), axis=1)
F = np.concatenate((F_top, F_bot))

# Husk at ændre delta t hvis det bliver nødvændigt
delta_t = 0.1
phi = np.eye(F.shape[0]) + delta_t*F

In [14]:
"""
    Her definereri vi resten af de parameter som vores filter skal bruge:
        phi, S_u, S_w, x_init_guess, M_init_guess, x
    og vi laver filteret
"""
S_u, S_w = np.eye(6), np.eye(6)
x_init, M_init = np.zeros(6), np.zeros((6, 6))

kf = Kalman(phi, S_u, S_w, x_init, M_init, x_state)

(6,)
(13629, 6)
(6, 6)
(13629, 6)


ValueError: shape mismatch: objects cannot be broadcast to a single shape